# RODEO2antiSMASH
## Settings

In [1]:
import os
import csv
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re
import multiprocessing as mp

### Directories with input files
`gb_dir` is a directory with GenBank files trimmed to the size of genomic region analyzed by RODEO. Names are {RODEO QUERY} + '.gbk'

`rod_dir` is a directory with RODEO output.

`rod_dir_type` indicates the structure of the RODEO output directory. This variable may be either `'RODEO'` or `'RIPPER'`, all other values are ignored. `'RODEO'` correspond to the "classic" output when the information about all BGCs is stored in a single CSV table `main_co_occur.csv`. `'RIPPER'` is used if the RODEO is a part of the RIPPER analysis. In this case every BGCs has its own subdirectory.

In [2]:
gb_dir = 'gbk/'         #Directory with trimed GenBank files
rod_dir = 'ripper/'      #Directory with RODEO output
rod_dir_type = 'RIPPER'  #Structure of the rod_dir directory

In [ ]:
def operon_borders_proteins():
    
           
    with open(rod_dir + x + '/main_co_occur.csv', 'r') as infile:
        infile.next()
            
        operon_accs = []
                
        prev_end = 0
        prev_strand = ''
        seed = x

        for row in csv.reader(infile):
            start = min(int(row[4]), int(row[5]))
            if (row[6] == prev_strand) and (start - prev_end < 100):
                operon_accs.append(row[3])
                    
            else:
                if seed in operon_accs:
                    return (operon_accs[0], operon_accs[-1])
                    
                operon_accs = [row[3]]
                    
            prev_end = max(int(row[4]), int(row[5]))
            prev_strand = row[6]
        
        if seed in operon_accs:
            return (operon_accs[0], operon_accs[-1])

In [ ]:
def get_bg_list(name, gotta_gbks):
    #dirs = ['/home/gyrase/datatank/Bikmetov/ycao_clusters/ripper_out/rodeo/rodout/' + x for x in os.listdir('/home/gyrase/datatank/Bikmetov/ycao_clusters/ripper_out/rodeo/rodout/') if x in ycaos]
    #dirs += ['rodout/' + x for x in os.listdir('rodout/') if x in ycaos]
    
    inlist = []
    with open('domfilter/' + name + '/' +name + '_table' +'.csv', 'r') as infile:
        inlist = [x for x in csv.reader(infile)]


    li = []

    for line in inlist:
        for dir in gotta_gbks:
            #print(dir + ' is processing')
            with open(rod_dir + dir + '/main_co_occur.csv', 'r') as infile:
                for row in csv.reader(infile):            
                    try:
                        if row[7] == line[0]:
                            li.append(row[3])
                    except:
                        pass
    return li

In [ ]:
def operon_borders_coords(operon_borders):
    operon_coords = {}
    prot_id_regexp = re.compile('[A-Z]{2}_[0-9]+\.[0-9]')
    
    
    for x in os.listdir(gb_dir):
        
        prot_id = x.rstrip('.gbk')
        
        start_id = operon_borders[prot_id][0]
        end_id = operon_borders[prot_id][1]
        
        operon_coords[prot_id] = []
        
        gb_record = SeqIO.parse(gb_dir + x, "genbank", generic_dna)
        for record in gb_record:
            for feature in record.features:
                
                if 'protein_id' in feature.qualifiers:
                    
                    if feature.qualifiers['protein_id'][0]  == start_id:
                        operon_coords[prot_id].append(feature.location.start + 1)# genbank is 1-based, python is 0-based
                           
                    if feature.qualifiers['protein_id'][0]  == end_id:
                       
                        operon_coords[prot_id].append(int(feature.location.end))
                
                elif 'pseudo' in feature.qualifiers:
                    
                    if 'inference' in feature.qualifiers:
                        
                        inference_prot_id_search = prot_id_regexp.search(feature.qualifiers['inference'][0])
                        
                        if inference_prot_id_search is not None:
                            
                            inference_prot_id = inference_prot_id_search.group(0)
                            
                            if inference_prot_id  == start_id:
                                operon_coords[prot_id].append(feature.location.start + 1)# genbank is [1:], python is

                            if inference_prot_id  == end_id:
                                operon_coords[prot_id].append(int(feature.location.end))
                        else:
                            
                            if feature.qualifiers['locus_tag'][0]  == start_id:
                                operon_coords[prot_id].append(feature.location.start + 1)# genbank is [1:], python is

                            if feature.qualifiers['locus_tag'][0] == end_id:
                                operon_coords[prot_id].append(int(feature.location.end))
    return operon_coords

In [ ]:
def antismash(name, types, operon_coords, gotta_gbks):
    
    inf = open('contig_edge.csv', 'r')
    contig_edge_list = [x[0] for x in csv.reader(inf)]
    inf.close
    
        
    bg_list = get_bg_list(name, gotta_gbks)
    #print bg_list
    
    bg = False
    
    contig_edge = False
    
    for x in os.listdir('trimmed_gbk_headers_corrected/'):
        
        with open('trimmed_gbk_headers_corrected/' + x, 'r') as infile:
            with open('trimmed_gbk_headers_as_improved/' + x, 'w') as outfile:
                
                prot_id = x.rstrip('.gbk')
                
                if prot_id in contig_edge_list:
                    contig_edge = True
                else:
                    contig_edge = False
                print prot_id
                cluster_start = operon_coords[prot_id][0]
                cluster_end = operon_coords[prot_id][1]
               
                for line in infile:
                    
                    if line.startswith('                     /protein_id='):
                        if line.split('\"')[-2] in bg_list:
                            bg = True
                    
                    elif line.startswith('                     /transl_table=') and bg:
                        outfile.write('                     /sec_met="Kind: biosynthetic"' + '\n')
                        bg = False
                    
                    outfile.write(line)
                    if line[0:8] == 'FEATURES':
                        outfile.write('     cluster         ' + '..'.join((str(cluster_start), str(cluster_end))) + '\n' + 
                                      '                     /contig_edge=\"' + str(contig_edge) + '\"' + '\n' +
                                      '                     /product=\"' + types[name] + '\"' + '\n')

In [36]:
def rodeo_output_iterator(rod_dir, rod_dir_type):
    
    if not rod_dir_type:
        if 'main_co_occur.csv' in os.listdir(rod_dir):
            rod_dir_type = 'RODEO'
        else:
            rod_dir_type = 'RIPPER'
    
    if rod_dir_type == 'RODEO':
        # It woulod be easier to use Pandas, but this way is more memory-efficient.
        # It may be crucial when working with large datasets
        with open('%s/main_co_occur.csv' % rod_dir) as infile:
            infile.next()
            prev_seed = None
            table = []
            for row in csv.reader(infile):
                
                if prev_seed is None:
                    prev_seed = row[0]
                
                if row[0] == prev_seed:
                    table.append(row)
                
                else:
                    yield table
                    prev_seed = row[0]
                    table = [row]
                
            yield table
            
    elif rod_dir_type == 'RIPPER':
        for folder in os.listdir(rod_dir):
            if 'main_co_occur.csv' in os.listdir('%s/%s' % (rod_dir, folder)):
                with open('%s/%s/main_co_occur.csv' % (rod_dir, folder)) as infile:
                    
                    infile.next()
                    yield list(csv.reader(infile))

In [37]:
gb_dir = 'gbk/'         #Directory with trimed GenBank files
rod_dir = 'rodeo/'      #Directory with RODEO output
rod_dir_type = 'RODEO'  #Structure of the rod_dir directory

In [38]:
t = rodeo_output_iterator(rod_dir, rod_dir_type)
t.next()

[['WP_005486705.1',
  'Streptomyces sp. SID5476',
  'NZ_WWIH01000042.1',
  'WP_028797007.1',
  '212271',
  '211668',
  '-',
  'PF01725',
  'Ham1p_like',
  'Ham1 family',
  '6.4e-62'],
 ['WP_005486705.1',
  'Streptomyces sp. SID5476',
  'NZ_WWIH01000042.1',
  'WP_005486686.1',
  '212680',
  '212305',
  '-',
  'PF04012',
  'PspA_IM30',
  'PspA/IM30 family',
  '1.9e-05',
  'PF07889',
  'DUF1664',
  'Protein of unknown function (DUF1664)',
  '0.00028',
  'PF06009',
  'Laminin_II',
  'Laminin Domain II',
  '0.00044',
  'PF12795',
  'MscS_porin',
  'Mechanosensitive ion channel porin domain',
  '0.00045'],
 ['WP_005486705.1',
  'Streptomyces sp. SID5476',
  'NZ_WWIH01000042.1',
  'WP_005486689.1',
  '213597',
  '212859',
  '-',
  'PF01138',
  'RNase_PH',
  "3' exoribonuclease family, domain 1",
  '9.1e-26',
  'PF03725',
  'RNase_PH_C',
  "3' exoribonuclease family, domain 2",
  '2.6e-10'],
 ['WP_005486705.1',
  'Streptomyces sp. SID5476',
  'NZ_WWIH01000042.1',
  'WP_005486691.1',
  '213921'

In [28]:
def test_generator(a):
    print 'kek'
    if a == 1:
        for i in range(5):
            yield i
            print 'lol'
        yield 'fjhfjf'
    else:
        for i in range(100, 105):
            yield i

In [29]:
t = test_generator(1)
print t

<generator object test_generator at 0x7f9b546e9eb0>


In [30]:
t.next()

kek


0

In [31]:
for x in t:
    print x

lol
1
lol
2
lol
3
lol
4
lol
fjhfjf


In [4]:
a = None
b = 1
if a:
    print 'a'
if b:
    print 'b'    

b


In [5]:
infile = open('rtrt', 'r')

IOError: [Errno 2] No such file or directory: 'rtrt'

In [ ]:
type(os.listdir)

In [ ]:
#os.mkdir('trimmed_gbk_headers_as')

In [ ]:
operon_borders = {x : table_reader(x) for x in gotta_gbks}

In [ ]:
operon_coords = get_operon_coords(operon_borders)

In [ ]:
#os.mkdir('trimmed_gbk_headers_as_improved/')

In [ ]:
make_antismash_great_again('tomms', {'tomms': 'thiopeptide'}, operon_coords, gotta_gbks)